#  <span style="color:darkblue"> Optimization </span>

<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Learning Objectives </span>

- Classify optimization problems as linear, quadratic, nonlinear, and mixed-integer problems.
- Learn how to interpret the solutions of optimization problems.
- Learn how to input optimization problems in Julia/JuMP).
- Assess the sensitivity of a solution with respect to constraints.
- Learn how to formulate and solve optimization problems for multiple examples.

<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Organization </span>

- First, we'll review some of the types of optimization problems.
- Next, we'll solve a Sudoku puzzle.
- You'll formulate and solve a simple blending and pooling problem.
- Lastly, we'll analyze the blending and pooling problem from a sensitivity perspective.
    
<hr style="border:6px solid black"> </hr>

## <span style="color:darkblue"> Problem Forms </span>

Optimizers generally exploit a specific mathematical structure in the problem formulation as a means to solve a problem. In fact, some underlying assumptions are always required to ensure a problem is optimized. One common example arrise from combining KKT conditions with constraint qualifications or alternatively regularity conditions for nonlinear programs.

These problem types range from linear programs for which problems with 100k variables and constraints may routinely be solved [in less than a minute](http://plato.asu.edu/ftp/lpsimp.html) to continuous convex problems to nonconvex nonlinear mixed-integer problems (were some problems with fewer than 5 variables have yet to be solved). As such, using an appropriate optimizer for a particular problem type can transform an unsolveable problem into a trival one. The most heavily studied and widely used descriptors for each program are linear, quadratic, nonlinear, and mixed-integer.

**Types of based on objectives/constraints:**

- **Linear:** All the constraints and the objective are linear.
- **Quadratic** all the constraints and the objective are quadratic or linear.
- **Nonlinear** at least one constraint or the objective is not linear.

**Types based on manner of variable used:**

- **Continuous:** All variables can vary between values in some possibly open interval.
- **Integer:** All variables are integer valued.
- **Mixed-integer:** The problem contains both integer and continuous variables.

One of the central focuses of optimization research has focused on developing specialized routines for solving programs with other special forms. A number of other important forms include [specially-ordered sets](https://link.springer.com/article/10.1007/BF01589393), [second-order cones](https://ieeexplore.ieee.org/abstract/document/6983691), [equilibrium constraints](https://faculty.wcas.northwestern.edu/~lchrist/Ferris_mathematical_programs2.pdf), as well as a myriad of special nonlinear forms. When attempting to solve a difficult model it's worth checking a few references (such as [Mosek's Modeling Cookbook](https://docs.mosek.com/MOSEKModelingCookbook-letter.pdf)) to see if the problem can be re-written as simplier form or a simplier problem type.

This trend has lead to the development of a number of specialized languages and software packages used to describe optimization problems interface with a myriad of different optimizers. In Python, the main tools for this are [Pyomo]() and [CVXOPT](https://cvxopt.org/). In Julia, we have [JuMP](https://jump.dev/JuMP.jl/stable/), and [Convex.jl](https://jump.dev/Convex.jl/stable/). A number of other commercial offerings available include [GAMS](https://www.gams.com/), [AIMMS](https://en.wikipedia.org/wiki/AIMMS), and [AMPL](https://ampl.com/) along with a number of offerings associated with commercially available solvers such as [CPLEX Optimization Studio by IBM](https://www.ibm.com/analytics/cplex-optimizer) or [Xpress Mosel for FICO Express](https://www.fico.com/en/products/fico-xpress-optimization). We'll work through formulating and solving a couple of optimization problems using JuMP in this workbook.

<hr style="border:2px solid gray"> </hr>

In [1]:
# Run this block once. Adds Julia packages to your Julia local installation from the package manager

using Pkg               # Import functions from the package manager into this session.
Pkg.add("JuMP")         # A modeling language for Mathematical optimization in Julia.
Pkg.add("Ipopt")        # A highly performant nonlinear optimizer.
Pkg.add("GLPK")         # An open-source linear and mixed-integer linear optimizer

nothing # suppresses long output on last line

   Updating registry at `C:\Users\wilhe\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\wilhe\Project.toml`
No Changes to `C:\Users\wilhe\Manifest.toml`


<hr style="border:2px solid gray"> </hr>

## <span style="color:darkblue">  Solving Sudoku with Mixed-Integer Programming* </span>

* Adapted from a now defunct example by Ian Dunning at https://www.juliaopt.org/notebooks/JuMP-Sudoku.html.

Sudoku is a popular number puzzle. The goal is to place the digits 1,...,9 on a nine-by-nine grid, with some of the digits already filled in. Your solution must satisfy the following rules:

    The numbers 1 to 9 must appear in each 3x3 square
    The numbers 1 to 9 must appear in each row
    The numbers 1 to 9 must appear in each column

Any combination satisfying the above rules is a valid solution. Thus, we're simply seeking any feasible solution. While this isn't strictly an optimization problem (it's a constraint satisfaction problem), but we can make it into equivalent optimization problem. Namely, we can just specify that the objective is some constant value of our choosing. 

We can model this problem using 0-1 integer programming: a problem where all the decision variables are binary. We'll use JuMP to create the model, and then we can solve it with any integer optimizer.

In [2]:
using JuMP, GLPK                    # import functions from the JuMP & GLPK package into your current environment

# create a model called "sudoku" which holds all variables, constraints, 
# functions, and options needed to solve the sudoku problem
sudoku = Model(GLPK.Optimizer)

# add binary variables
@variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)   # Bin indicates the variable is binary (has a value of 0 or 1)

nothing

Only one digit can be in each square. So for each square represented by a pair of i,j variables the sum over k should be equal to one.

In [3]:
for i = 1:9, j = 1:9  
    @constraint(sudoku, sum(x[i,j,k] for k=1:9) == 1)
end

Each variable can only appear once in each column.

In [4]:
for j = 1:9, k = 1:9
    @constraint(sudoku, sum(x[i,j,k] for i=1:9) == 1)
end

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Each variable can only appear once in each row. Fill in the appropriate expression.
</div>

In [5]:
for ind = 1:9, k = 1:9
     @constraint(sudoku, sum(x[i,ind,k] for i=1:9) == 1)
end

Each variable can only appear once in each 3-by-3 subgrid.

In [6]:
# i is the top left row, j is the top left column
# We'll sum from i to i+2, e.g. i=4, r=4, 5, 6
for i = 1:3:7, j = 1:3:7, k = 1:9
    @constraint(sudoku, sum(x[r,c,k] for r=i:i+2, c=j:j+2) == 1)
end

Fills in the provided values using zero to represent an empty cell

In [7]:
# enter the initial values
init_val = [ 5 3 0   0 7 0   0 0 0;
             6 0 0   1 9 5   0 0 0;
             0 9 8   0 0 0   0 6 0;
    
             8 0 0   0 6 0   0 0 3;
             4 0 0   8 0 3   0 0 1;
             7 0 0   0 2 0   0 0 6;
    
             0 6 0   0 0 0   2 8 0;
             0 0 0   4 1 9   0 0 5;
             0 0 0   0 8 0   0 7 9]

# if the value is specified initially add a constraint fixing the variables to that value.
for i = 1:9, j = 1:9
    if init_val[i,j] != 0
        @constraint(sudoku, x[i,j,init_val[i,j]] == 1)
    end
end

<div class="alert alert-block alert-info">
<b>What type of optimization problem was the sudoku? Is GLPK an appropriate optimizer for this type of problem?</b> You'll want to search for GLPK's documentation and check there.
</div>

<hr style="border:2px solid gray"> </hr>

In [8]:
optimize!(sudoku)

x_val = value.(x) # Extract the values of x

9×9×9 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.

To display the solution, we need to look for the values of **x**[i, j, k] that are equal to 1.

<div class="alert alert-block alert-warning">
<b>INTERACTIVE! Unpack the solution to a 9-by-9 matrix and confirm this is a valid Sudoku solution.</b> Note that the 
solution returned may not be an integer. So you'll need to round this to the nearest integer using <b>iround(y)</b>. 
</div>

In [9]:
final_val = zeros(9,9)

for i = 1:9, j = 1:9, k = 1:9
    if x_val[i,j,k] != 0
        final_val[i,j] = k
    end
end

final_val

9×9 Array{Float64,2}:
 5.0  3.0  2.0  2.0  7.0  6.0  1.0  3.0  7.0
 6.0  4.0  7.0  1.0  9.0  5.0  9.0  5.0  8.0
 1.0  9.0  8.0  3.0  4.0  8.0  4.0  6.0  2.0
 8.0  1.0  9.0  9.0  6.0  1.0  5.0  4.0  3.0
 4.0  5.0  3.0  8.0  5.0  3.0  7.0  9.0  1.0
 7.0  2.0  6.0  7.0  2.0  4.0  8.0  2.0  6.0
 2.0  6.0  1.0  6.0  3.0  2.0  2.0  8.0  4.0
 3.0  8.0  5.0  4.0  1.0  9.0  6.0  1.0  5.0
 9.0  7.0  4.0  5.0  8.0  7.0  3.0  7.0  9.0

<hr style="border:2px solid gray"> </hr>

## <span style="color:darkblue"> Integrating water system treatment in the process industry </span>

* Adapted from [R. Karuppiah & I. Grossmann. Global optimization for the synthesis of integrated water systems in chemical processes. Computers and Chemical Engineering 30 (2006) 650–673](https://www.sciencedirect.com/science/article/abs/pii/S0098135405002991)

The efficient utilitization of water will be an integral part to solving the numerous water management issues of today and the near future. Increasing usage of water, anticipated water scarities, and changing regulations are expected to result in numerous related challenges in the next  decades. These challenges may extend to the adaptation of numerous industrial processes. as fresh water is a key resource in such processes.  These include washing operations in the food processing, thermal processing (cooling, quenching, and scrubbing) in iron and steel  manufacturing, and the desalination of crude oil. In each of this  processes, some level of contamination is introduced into the freshwater by the process. This process water is then treated in a central  facility to remove contaminants and satisfy local regulatory specifications for the disposal of wastewater. In design or improvement of these central facilities, three key questions must be considered. 1) What unit operations need to be included? 2) How should these process units  be connected? 3) How should one operate these processes units?

A typical design objective for a wastewater treatment plant may be to minimize it's overall cost in a given year of operation. That is we minimize the sum of the cost fo freshwater intake, the investment cost associated with each treatment unit, and the operating cost of  each unit. This can be achieved by minimizing the follow objective:

$\Phi = A_R\sum_{j} C_j^{Inv} F_j^{\alpha} + H\sum_{j} F_j C_j^{Op} + H F_{fw} C_{fw}$ 

where H is the number of hours of operation per annum ($h$); the cost of freshwater is given by $C_{fw}$ (`$/ton`); the fresh water intake mass if given by $F_{fw}$ (`ton/h`); The annualized factor for investment on treatment units is given by $A_R$; the $C_j^{Inv} F_j^{\alpha}$ is the investment cost (`$`) of the $j$-th treatment unit which treats a flowrate stream of contaminated water $F_j$ (`ton/h`); $C_j^{Op} F_j^{\alpha}$ is the operating cost (`$/h`) of the $j$-th treatment unit which treats a flowrate of contaminated water $F_j$ (`ton/h`); the parameter $\alpha$ is a constant cost function exponent $0 \leq \alpha \leq 1$.

The **mixers** units can be described as follows:
- The outlet flowrate is the sum of the inlet stream flow rate: $F^{outlet} = \sum_{i = 1}^{n} F^{inlet}_i $ where $F^{outlet}$, $F^{inlet}_1$, $\ldots$, $F^{inlet}_n$ are given in (tons/hr) and $n$ is the number of inlets streams.
- The mass of each contaminant in the outlet equals the mass of each contaminant in the inlet streams: $F^{outlet}C_j^{outlet} = \sum_{i = 1}^{n} F^{inlet}_i C_j^{inlet}$ 

The **splitter** units:
- The outlet flowrates are the sum of the inlet flowrate: $F^{inlet} = \sum_{i = 1}^{m} F^{outlet}_i $ where $F^{inlet}$, $F^{outlet}_1$, $\ldots$, $F^{outlet}_m$ are given in (tons/hr) and $m$ is the number of outlet streams.
- The each outlet streams contaminant concentration is equal to the inlet stream contaminant concentration: $F^{outlet}_m$. 

A fixed mass of contaminant is added to a stream by each **process unit**:
- A process unit has a single inlet stream and a single outlet stream: $F^{inlet} = F^{outlet}$
- The process unit reduces the l $F^{inlet}C_j^{inlet} + L^{p}_j = F^{outlet}C_j^{outlet}$

The **treatment units**:
- A process unit has a single inlet stream and a single outlet stream with the same flowrate: $F^{inlet} = F^{outlet}$
- The concentration of contaminant in the outlet $C_j^{outlet} = \beta^{t}_j C_j^{inlet}$ where $\beta^{t}_j$ is the removal ratio of $j$.

Two contaminants A & B are introduced into the system and the concentration of each contaminant must be reduced to 10ppm prior to discharging it into the enviroment. The cost of freshwater is assumed to be $1/ton, the annualized factor for investment on the treatment unit is taken to be 0.1, and the total time of operation of the plant is 8000h/year. While numerous different treatment technologies and separation train configurations are possible, we've settled on using two distinct units to perform a primary and secondary treatment. We'd like to determine whether the inclusion of a recycle stream(s) may reduce the process cost and whether is preferable to have the recyle stream bridge one unit or two.

<img src="Treatment_Train.png" width="800">

The flowrate of water required by each process unit, along with  containment inlet concentration limits, and amount of contaminant  discharged are given in Table 1. We'll assume an 𝛼 value of 0.7, an investment. The process unit uses 40 ton/h of freshwater and discharges 1 kg/h of A and 1kg/h of B.

The efficiency of the treatment units are given by the removal ratio in Table 2 along with cost model constants: 

<img src="SpecTable1.png" width="500">

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b>  Is this model convex or nonconvex? Continuous or mixed-integer? Why? 
</div>

We've now started to define objective and constraint functions which compute the concentration and flow rates associated with each stream and the overall system cost. Generally, there are two approaches to doing this. We can introduce a new variable for each concentration and flowrate and allow the optimizer to enforce mass balances. Alternative, we can algebraiclly re-arrange these mass balances and avoid introducing a large number of variables. In some cases, the former approach exposes special forms that an optimizer may use effectively. In other cases, the decreased number of variables may be more beneficial. We primarily use the later approach but we'll introduce variables to handle the recycle-loops which generally cannot be re-arranged into an explicit algebraic form. The optimization variables are $x=\left(F_{\mathrm{feed}} ,F_2 ,F_9 ,C_{A,4} ,C_{B,4} ,C_{A,9} ,C_{B,9} ,F_6 ,C_{A,8} ,C_{B,8} \right)$. Take some time to inspect the constraint function and reflect on the choice of these variables.

<div class="alert alert-block alert-info">
Generally optimizers require that all variables have both lower (lb) and upper (ub) bounds defined. If there aren't any obvious choices in a problem, you may have to exercise your best judgement and choose reasonable bounds. Equipment based bounds on flowrate are provided below. Further, we assume 0.1 is a sufficiently high concetration such that it will be an inactive constraint. Furthermore, an initial guess (x0) often must be specified. For convex problems, the solution obtained will generally not depend on this guess. 
</div>

In [10]:
# Bounds
lb = [0.1; 0.1; 0.1; 0.0; 0.0; 0.0; 0.0; 0.1; 0.0; 0.0]
ub = [100.0; 100.0; 100.0; 0.1; 0.1; 0.1; 0.1; 100.0; 0.1; 0.1]

# Specify initial guess
x0 = 0.5*(lb + ub)

nothing

We start by creating an optimization problem, assign it the Ipopt optimizer, and add variables to the model.

In [11]:
using JuMP, Ipopt

model = Model(Ipopt.Optimizer)

@variable(model, lb[i] <= x[i=1:10] <= ub[i], start = x0[i])

10-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b>  Finish the intermediate calculations and constraints used to define the problem.
</div>

In [12]:
# process unit #1
Lp11 = 1.5/1000
Lp21 = 1/1000

@NLexpression(model, f1e, 40 + x[1])
@NLexpression(model, c1_1e, Lp11/f1e)
@NLexpression(model, c2_1e, Lp21/f1e)

# mixer #1
@NLexpression(model, f4e, x[2] - f1e - x[3]) 
@NLexpression(model, c1_2e, (c1_1e*f1e + x[4]*f4e + x[6]*x[3])/x[2])
@NLexpression(model, c2_2e, (c2_1e*f1e + x[5]*f4e + x[7]*x[3])/x[2])

# treatment unit #1
@NLexpression(model, f3e, x[2])
@NLexpression(model, c1_3e, (1.0 - 0.95)*c1_2e)
@NLexpression(model, c2_3e, (1.0 - 0.2)*c2_2e)

# splitter #1
@NLexpression(model, f5e, f3e - f4e)
@NLexpression(model, c1_5e, c1_3e)
@NLexpression(model, c2_5e, c2_3e)

@NLconstraint(model, ceq1, x[4] - c1_3e == 0.0)
@NLconstraint(model, ceq2, x[5] - c2_3e == 0.0)

# mixer #2
@NLexpression(model, f6e, x[8])
@NLexpression(model, f8e, f6e - f5e)
@NLexpression(model, c1_6, (c1_5e*f5e + x[9]*f8e)/f6e)
@NLexpression(model, c2_6, (c2_5e*f5e + x[10]*f8e)/f6e)

# treatment unit #2
@NLexpression(model, f7e, f6e)
@NLexpression(model, c1_7e, (1.0 - 0.1)*c1_6)
@NLexpression(model, c2_7e, (1.0 - 0.7)*c2_6)

# splitter #2
@NLexpression(model, f10e, f7e - f8e - x[3])
@NLexpression(model, c1_10, c1_7e)
@NLexpression(model, c2_10, c2_7e)

@NLconstraint(model, ceq3, x[9] - c1_7e == 0)
@NLconstraint(model, ceq4, x[10] - c2_7e == 0)
@NLconstraint(model, ceq5, x[6] - c1_7e == 0)
@NLconstraint(model, ceq6, x[7] - c2_7e == 0)

# inequality effluent constraints
c1_spec = 0.00001;
c2_spec = 0.00001;
@NLconstraint(model, cons_ef1, c1_10 <= c1_spec)
@NLconstraint(model, cons_ef2, c2_10 <= c2_spec)

nothing

In [13]:
# financial assumptions
AR = 0.1                            # annualized factor
H = 8000                            # total hours plant operates
alpha = 0.7                         # scaling term for capital cost
Cwater = 1.0                        # cost of freshwater ($/ton)
Cinv1, Cinv2 = 16800.0, 12600.0     # investment costs
Cop1,  Cop2 = 0.0067, 1.0           # marginal operating costs
    
@NLobjective(model, Min, AR*(Cinv1*x[2]^alpha + Cinv2*x[8]^alpha) + Cop1*x[2] + Cop2*x[8] + H*x[1]*Cwater)

Now, we solve the optimization problem.

In [ ]:
# Solve optimization problem
JuMP.optimize!(model)

# Retrieve solution information
println("Result Status: ", primal_status(model))
println("Termination Status: ", termination_status(model))
println("Objective values: ", objective_value(model))

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Next we extract information about the optimization  problem. We'll want to check the status of the solve and the objective value, initially. Generally, optimizers will return codes to indicate if the solve was successful and to indicate any potential numerical issues that occur. Now, check the optimal solution and describe the system configuration.
</div>

In some cases, manufacturing plants are permitted a certain allowable amount of emissions at a particular site. Multiple emission sources may exist on a single site and it can be valuable to understand how expensive it is to reduce emissions in each process unit on that site. Let's determine the profitability associated with potentially relaxing our emissions target for this process. In order to do this, we'll perform a sensitivity analysis. That is to say, how much will changing a constraint and/or the variable bounds result in changing the solution? If we can perturb a constraint without affect this solution then the constraint is inactive. Otherwise, the dual values indicate the degree to which the solution may change by altering a constraint. We can obtain this information from the lambda structure. Nonlinear inequality constraint dual values are accessed via as shown below:

In [ ]:
println("Effluent constraint #1 & #2 duals = $(dual(cons_ef1)) & $(dual(cons_ef2))")

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> How much effect would changing the 10ppm effluent specification slightly have on the process cost? 
</div>

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Based on the form of the optimization problem, would you expect a local (convex) optimizer to return the best possible objective value? Why or why (not)? What sort of numerical experiments  would support a determination in either case?
</div>

<hr style="border:6px solid black"> </hr>

# A question to reflect on
We've solved two optimization problems: an integer program and a  nonlinear program. The full design of a wastewater treatment plant may  be handled by combining these two approaches. This can include which  process units, how many process units, and treatment units are included  as well as a full exploration of recycle loop configurations. This  variety of problem is termed superstructure optimization and developing efficient means of solving these problems remains an active area of research. How might we modify the above problem using 0-1 integer variables to explore other potential recycle configurations?

<hr style="border:6px solid black"> </hr>